In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from scipy.stats import poisson, uniform, expon, pareto
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

In [152]:
p0, p0p, p1, p0pp = sp.symbols('p_0 p_{00} p_1 p_{000}')
a, b, x, g, p, E  = sp.symbols('a b x g p E')

In [153]:
p0fail = (1-p)*p1*sp.exp(-x) / (a*p*(1-sp.exp(-b+E))+a*(1-p)*(1-sp.exp(-b)))
p0fail

p_1*(1 - p)*exp(-x)/(a*p*(1 - exp(E - b)) + a*(1 - p)*(1 - exp(-b)))

In [154]:
f0failHi = a
f0failLo = a*sp.exp(-b+x)+a**2*(1-sp.exp(-b+x)) + a*(1-a)*(1-sp.exp(-b+x))*(a+(1-a)*g)
f0failLo

a**2*(1 - exp(-b + x)) + a*(1 - a)*(1 - exp(-b + x))*(a + g*(1 - a)) + a*exp(-b + x)

In [137]:
p1_solved = 1/(1/(a*p*(1-sp.exp(-b+E)) + a*(1-p)*(1-sp.exp(-b))) + 1 + (1-a)/(1-2*a))
p1_solved

1/(1 + 1/(a*p*(1 - exp(E - b)) + a*(1 - p)*(1 - exp(-b))) + (1 - a)/(1 - 2*a))

In [149]:
p1_solved.evalf(subs={p:0}).evalf(subs={a:0.3, b:1.})

0.124637683091749

loIntegrandFail = sp.simplify(p0fail.evalf(subs={p1:p1_solved}) * f0failLo)
loIntegralFail  = sp.simplify(sp.integrate(loIntegrandFail, (x, 0, b)))

In [155]:
loIntegrandFail = sp.simplify(p0fail * f0failLo)
loIntegralFail  = sp.simplify(sp.integrate(loIntegrandFail, (x, 0, b)))

In [157]:
sp.simplify(loIntegralFail.evalf(subs={p:0, a:0.3, b:1.}))

p_1*(0.20483141363403*g + 0.79516858636597)

hiIntegrandFail = sp.simplify(p0fail.evalf(subs={p1:p1_solved}) * f0failHi)
hiIntegralFail  = sp.simplify(sp.integrate(hiIntegrandFail, (x, b, sp.oo)))

In [158]:
hiIntegrandFail = sp.simplify(p0fail * f0failHi)
hiIntegralFail  = sp.simplify(sp.integrate(hiIntegrandFail, (x, b, sp.oo)))

In [160]:
sp.simplify(hiIntegralFail.evalf(subs={p:0, a:0.3, b:1.}))

0.581976706869326*p_1

In [140]:
p0success = p*p1*sp.exp(-x+E) / (a*p*(1-sp.exp(-b+E))+a*(1-p)*(1-sp.exp(-b)))
p0success

p*p_1*exp(E - x)/(a*p*(1 - exp(E - b)) + a*(1 - p)*(1 - exp(-b)))

In [118]:
f0successHi = a
f0successLo = a*sp.exp(-b+E+x)+a**2*(1-sp.exp(-b+E+x)) + a*(1-a)*(1-sp.exp(-b+E+x))*(a+(1-a)*g)
f0successLo

a**2*(1 - exp(E - b + x)) + a*(1 - a)*(1 - exp(E - b + x))*(a + g*(1 - a)) + a*exp(E - b + x)

In [119]:
loIntegrandSuccess = sp.simplify(p0success.evalf(subs={p1:p1_solved}) * f0successLo)
loIntegralSuccess  = sp.simplify(sp.integrate(loIntegrandSuccess, (x, 0, b-E)))

In [120]:
hiIntegrandSuccess = sp.simplify(p0success.evalf(subs={p1:p1_solved}) * f0successHi)
hiIntegralSuccess  = sp.simplify(sp.integrate(hiIntegrandSuccess, (x, b-E, sp.oo)))

In [121]:
full = loIntegralFail + hiIntegralFail + loIntegralSuccess + hiIntegralSuccess + 3*a - a**2 + 2*a**2/(1-2*a)
full

-a**2 + 2*a**2/(1 - 2*a) + 3*a + Piecewise((a*(0.666666666666667*a*p - 0.666666666666667*a - 0.333333333333333*p + 0.333333333333333)/(1.0*a**2*p*exp(E) - 1.0*a**2*p - 1.0*a**2*exp(b) + 1.0*a**2 - 0.666666666666667*a*p*exp(E) + 0.666666666666667*a*p - 0.666666666666667*a + 0.333333333333333*exp(b)), Ne(1.0*a**2*p*exp(E) - 1.0*a**2*p - 1.0*a**2*exp(b) + 1.0*a**2 - 0.666666666666667*a*p*exp(E) + 0.666666666666667*a*p - 0.666666666666667*a + 0.333333333333333*exp(b), 0)), (Piecewise((nan, Eq(a*(2*a*p - 2*a - p + 1)/(3*a**2*p*exp(E) - 3*a**2*p - 3*a**2*exp(b) + 3*a**2 - 2*a*p*exp(E) + 2*a*p - 2*a + exp(b)), 0)), (a*(-b*(0.666666666666667*a*p - 0.666666666666667*a - 0.333333333333333*p + 0.333333333333333)*(3*a**2*p*exp(E) - 3*a**2*p - 3*a**2*exp(b) + 3*a**2 - 2*a*p*exp(E) + 2*a*p - 2*a + exp(b))*Abs(a*(2*a*p - 2*a - p + 1)/(3*a**2*p*exp(E) - 3*a**2*p - 3*a**2*exp(b) + 3*a**2 - 2*a*p*exp(E) + 2*a*p - 2*a + exp(b))) + oo*(2*a*p - 2*a - p + 1)*(1.0*a**2*p*exp(E) - 1.0*a**2*p - 1.0*a**2*exp(b)

In [164]:
full = loIntegralFail + hiIntegralFail + (3*a - a**2 + 2*a**2/(1-2*a))*p1
full

p_1*(-a**2 + 2*a**2/(1 - 2*a) + 3*a) + Piecewise((p_1*(p - 1)/(p*exp(E) - p - exp(b) + 1), Ne(-p*exp(E) + p + exp(b), 1)), (Piecewise((nan, Eq(p_1*(p - 1)/(p*exp(E) - p - exp(b) + 1), 0)), (p_1*(p - 1)*(-b*Abs(p_1*(p - 1)/(p*exp(E) - p - exp(b) + 1)) + oo)/((p*exp(E) - p - exp(b) + 1)*Abs(p_1*(p - 1)/(p*exp(E) - p - exp(b) + 1))), True)), True)) + Piecewise((p_1*(-a**2*g*p + a**2*g + a**2*p - a**2 + 2*a*g*p - 2*a*g - 2*a*p + 2*a - b*(a**2*g*p - a**2*g - a**2*p + a**2 - 2*a*g*p + 2*a*g + 2*a*p - 2*a + g*p - g - p + 1) - g*p + g + (a**2*g*p - a**2*g - a**2*p + a**2 - 2*a*g*p + 2*a*g + 2*a*p - 2*a + g*p - g)*exp(b))/(p*exp(E) - p - exp(b) + 1), Ne(-p*exp(E) + p + exp(b), 1)), (b*p_1*(p - 1)/(p*exp(E) - p - exp(b) + 1), True))

In [172]:
full.evalf(subs={p1:p1_solved}).evalf(subs={a:0.3, b:.1, g:0.0, p:0.})

0.310876984825722

In [170]:
temp_ours        = full.evalf(subs={p:0})
temp_ours_lambda = sp.lambdify((a, b, g), temp_ours)
temp_ours_lambda

<function _lambdifygenerated(a, b, g)>

In [171]:
temp_ours_lambda(a=0.3, b=1., g=0.0)

2.6371452932353*p_1